<a href="https://colab.research.google.com/github/MarcosP7635/Energy/blob/main/Power_density_po_208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import astropy 
from astropy import constants as const
from astropy import units as units
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import urllib.request

def lc_read_csv(url):
    req =urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

In [ ]:
# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"

In [ ]:
# load data into a dataframe 

# this loading might not work, then use the line below
#df = pd.read_csv(livechart + "fields=decay_rads&nuclides=241am&rad_types=g")
df = lc_read_csv(livechart + "fields=decay_rads&nuclides=208po&rad_types=a")

#df = df[pd.to_numeric(df['intensity'],errors='coerce').notna()] # remove blanks (unknown intensities)
#df.intensity = df['intensity'].astype(float) # convert to numeric. Note how one can specify the field by attribute or by string 

#fig = px.scatter(df, x="energy", y="intensity", log_y=True) # plot in log scale
#fig.show()
df

,energy,unc_en,intensity,unc_i,daughter_level_energy,hindrance_factor,unc_hf,p_z,p_n,p_symbol,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,4220.0,15.0,0.00024,0.000070,899.166,0.62,0.19,84,124,Po,...,99.996,0.0004,5215.7,13,82,122,Pb,1-Oct-2009,C.J. CHIARA and F.G. KONDEV,2022-09-01
1,5114.9,1.4,99.99576,0.000406,0.000,1.00,,84,124,Po,...,99.996,0.0004,5215.7,13,82,122,Pb,1-Oct-2009,C.J. CHIARA and F.G. KONDEV,2022-09-01


In [ ]:
decay_energy = df["energy"][1] * units.keV

In [ ]:
half_life = df['half_life'][1] * units.year

In [ ]:
half_life

<Quantity 2.898 yr>

In [ ]:
half_life *= 1/np.log(2)

In [ ]:
half_life = half_life.to(units.second)

In [ ]:
atoms_per_s = 1 * units.mol / half_life 
atoms_per_s *= const.N_A
atoms_per_s

<Quantity 4.56429825e+15 1 / s>

In [ ]:
power_per_mol = (decay_energy * atoms_per_s).to(units.W)

In [ ]:
specific_power = power_per_mol / (208 * units.g)
specific_power

<Quantity 17.98283756 W / g>

In [ ]:
def mass_per_area(density, thickness): 
  return (density * thickness).to(units.g / units.cm**2)

unit_type = astropy.units.quantity.Quantity 

class Electric_battery:
    
    def __init__(self, conversion_type, properties_dict):
        self.conversion_type = conversion_type
        for key in properties_dict.keys():
            setattr(self, key.replace(" ", "_"), properties_dict[key])

    def evaluate(self, isotope_specific_power, isotope_mass_density):
        total_volume = (self.total_thickness * self.area).to(units.cm ** 3)
        isotope_mass_per_area = mass_per_area(isotope_mass_density, 
                                self.isotope_thickness)
        total_mass_per_area = self.mass_per_area_no_fuel + isotope_mass_per_area
        total_mass_density = (total_mass_per_area * units.cm**2 / 
                                total_volume)
        isotope_power_per_area = (isotope_mass_per_area * 
                                    isotope_specific_power)
        self.specific_power = (self.conversion_efficiency * 
                            isotope_power_per_area / 
                            total_mass_per_area)
        self.power_density = self.specific_power * total_mass_density
        return (self.power_density, self.specific_power)

In [36]:
aqueous_conversion_dict = {"isotope thickness" : 1 * units.micron, 
        "mass per area no fuel" : 0.4026
 * units.g / units.cm**2 ,
        "total thickness" : 2003.05 * units.micron,
        "area" : 1 * units.cm**2,
        "conversion efficiency" : .5}
direct_conversion_dict = {"isotope thickness" : .2 * units.micron,
    "mass per area no fuel" : .47064895 * units.g / units.cm**2,
    "total thickness" : 4.1665 * units.millimeter, "area" : 1 * units.cm**2,
    "conversion efficiency" : .2}
conversion_properties_dict = {"direct" : direct_conversion_dict,
                     "aqueous" : aqueous_conversion_dict}

In [37]:
battery_type = "aqueous"
a = Electric_battery(battery_type, 
conversion_properties_dict[battery_type])

In [38]:
a.isotope_thickness

<Quantity 1. micron>

In [39]:
a.evaluate(specific_power, 9.389 * units.g / units.cm**3)

(<Quantity 0.04214594 W / cm3>, <Quantity 0.02092002 W / g>)

Now consider the aqueous battery  but instead the water consists of T$_2$O

"water (1 mm)/Pt (50 nm)/TiO2 (1 mm)/glass (1 mm) structure"

In [34]:
titanium_thickness = 2 * units.micron
platinum_film_thickness = 50 * units.nanometer 
water_thickness = 1 * units.mm
glass_thickness = 1 * units.mm
glass_density = 3 * units.g/(units.cm**3)
titanium_density = 5 * units.g /(units.cm ** 3) 
platinum_density = 20 * units.g / (units.cm ** 3)
water_density = 1 * units.g / units.cm**3

In [35]:
total_mass_per_area = (mass_per_area(platinum_density, platinum_film_thickness)
                    + .5 * mass_per_area(platinum_density, titanium_thickness)
                    + .5 * mass_per_area(titanium_density, titanium_thickness)
+mass_per_area(glass_density, glass_thickness)
+mass_per_area(water_density, water_thickness))
total_mass_per_area

<Quantity 0.4026 g / cm2>

In [ ]:
titanium_thickness + platinum_film_thickness + glass_thickness + water_thickness

<Quantity 2002.05 micron>

In [ ]:
tritium_water_dict = {
"isotope thickness" : 1 * units.mm,
"mass_per_area_no_fuel" : .0029 * units.g/(units.cm**2),
"total_thickness" : 1003.05 * units.micron,
"area" : 1 * units.cm**2,
"conversion efficiency" : .5}

In [ ]:
battery = Electric_battery("tritium water",
tritium_water_dict)
battery.evaluate(power/units.g / 18, water_density)

(<Quantity 0.03157709 W / cm3>, <Quantity 0.02649016 W / g>)